In [4]:
import lmql
from pprint import pprint
import nest_asyncio
nest_asyncio.apply()

In [2]:
@lmql.query
def chain_of_thought(question):
    '''lmql
    # Q&A prompt template
    "Q: {question}\n"
    "A: Let's think step by step.\n"
    "[REASONING]"
    "Thus, the answer is:[ANSWER]."

    # return just the ANSWER to the caller
    return ANSWER
    '''

print(chain_of_thought('Today is the 12th of June, what day was it 1 week ago?'))


 5th of June.


In [5]:
@lmql.query
def chain_of_thought(review):
    '''lmql
    argmax
        
        """Review: {review}
        Q: What is the underlying sentiment of this review and why?
        A:[ANALYSIS]""" where not "\n" in ANALYSIS
        
        "Based on this, the overall sentiment of the message can be considered to be[CLS]" where CLS in [" positive", " neutral", " negative"]
        
        if CLS == " positive":
            "What is it that they liked about their stay? [FURTHER_ANALYSIS]"
        elif CLS == " neutral":
            "What is it that could have been improved? [FURTHER_ANALYSIS]"
        elif CLS == " negative":
            "What is it that they did not like about their stay? [FURTHER_ANALYSIS]"
    where
    STOPS_AT(FURTHER_ANALYSIS, ".")
    '''

review = """We had a great stay. Hiking in the mountains 
                    was fabulous and the food is really good."""

pprint(chain_of_thought(review))


c:\Users\predator\anaconda3\envs\gen-ai-and-llm-integration\lib\site-packages\lmql\runtime\bopenai\batched_openai.py:752: OpenAILogitBiasLimitationWarning: the required logit_bias is too large to be handled by the OpenAI API and will be limited to the first 300 tokens. This can lead to the violation of the provided constraints or undesired model output. To avoid this use less broad or no constraints.
  warnings.warn("the required logit_bias is too large to be handled by the OpenAI API and will be limited to the first 300 tokens. This can lead to the violation of the provided constraints or undesired model output. To avoid this use less broad or no constraints.", category=OpenAILogitBiasLimitationWarning)


LMQLResult(prompt='Review: We had a great stay. Hiking in the mountains \n'
                  '                    was fabulous and the food is really '
                  'good.\n'
                  'Q: What is the underlying sentiment of this review and '
                  'why?\n'
                  'A: The underlying sentiment of this review is positive '
                  'because the reviewer had a great stay, enjoyed hiking in '
                  'the mountains, and found the food to be really good.Based '
                  'on this, the overall sentiment of the message can be '
                  'considered to be positiveWhat is it that they liked about '
                  'their stay? \n'
                  'The reviewer liked the hiking in the mountains and the '
                  'food.',
           variables={'ANALYSIS': ' The underlying sentiment of this review is '
                                  'positive because the reviewer had a great '
                                